In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#%cd /content/drive/MyDrive/Colab Notebooks/data
#!ls

/content/drive/MyDrive/Colab Notebooks/Task5
labels.csv  reviews1.csv  reviews.csv  Untitled0.ipynb


In [ ]:
pd.set_option('display.max_columns', None)
df_rev = pd.read_csv('reviews.csv', delimiter=',', decimal=',', encoding='UTF-8')
df_rev.info()
df_rev.head()
df_rev.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      254 non-null    int64 
 1   text    254 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.1+ KB


(254, 2)

In [ ]:
pd.set_option('display.max_columns', None)
df_leb = pd.read_csv('labels.csv', delimiter=',', decimal=',', encoding='UTF-8')
df_leb.info()
df_leb.head()
df_leb.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         254 non-null    int64 
 1   sentiment  254 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.1+ KB


(254, 2)

In [ ]:
df = pd.merge(df_leb, df_rev, on='id', how='inner')
df.info()
df.head()
df.shape




<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 0 to 253
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         254 non-null    int64 
 1   sentiment  254 non-null    object
 2   text       254 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.9+ KB


(254, 3)

In [ ]:
x = df['text']
y = df['sentiment']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.7, random_state=42)


In [ ]:
nltk.download('stopwords')
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus = []
ids = []
for i in range(len(x_train)):
    review = re.sub('[^a-zA-Z]', ' ', x_train.iloc[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in all_stopwords]
    review = ' '.join(review)
    corpus.append(review)
    ids.append(x_train.index[i])

In [ ]:
corpus

In [ ]:
vectorizer = CountVectorizer()
x_train_vec = vectorizer.fit_transform(corpus)

# Naive Bayes
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(x_train_vec, y_train)

x_test_vec = vectorizer.transform(x_test)
y_pred = naive_bayes_model.predict(x_test_vec)

confusion = confusion_matrix(y_test, y_pred)

print("Матриця очікувань:")
print(confusion)

accuracy = accuracy_score(y_test, y_pred)

print(f"Точність моделі на тестовому наборі: {accuracy * 100:.2f}%")

Матриця очікувань:
[[60 12]
 [10 96]]
Точність моделі на тестовому наборі: 87.64%


In [ ]:
rev_res_df = pd.read_csv('reviews.csv')
rev = rev_res_df['text']

nltk.download('stopwords')
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

corpus = []
ids = []
for i in range(len(rev)):
    review = re.sub('[^a-zA-Z]', ' ', rev.iloc[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in all_stopwords]
    review = ' '.join(review)
    corpus.append(review)
    ids.append(rev.index[i])



rev = vectorizer.transform(corpus)
rev_pred = naive_bayes_model.predict(rev)
results_df = pd.DataFrame({'id': rev_res_df['id'], 'label': label_encoder.inverse_transform(rev_pred)})
results_df.to_csv('classified_reviews.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
